# Task 9 (Alternativo): Caching inteligente en herramientas (tools)

## Objetivo
Aprender a añadir **caching inteligente** a una herramienta para agentes, de forma que:
- La **primera llamada** sea lenta (simula API/cálculo costoso)
- Las **siguientes llamadas equivalentes** sean rápidas (cache hit)
- Tengamos **TTL** (caducidad), **métricas** (hits/misses) y **observabilidad** por consola

### Por qué merece la pena contarlo
En producción, la mayor parte del coste/latencia suele estar en:
- llamadas a APIs / BBDD
- herramientas costosas
- o prompts repetidos.

Un caché bien diseñado reduce **latencia**, **coste** y **tasa de errores** (timeouts / rate limits).

In [ ]:
import os
import time
import json
import hashlib
from typing import Annotated
from pydantic import Field
from dotenv import load_dotenv
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

load_dotenv()
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_id = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("=" * 80)
print("✅ Entorno configurado")
print(f"📍 Modelo: {model_id}")
print("=" * 80)

✅ Entorno configurado
📍 Modelo: gpt-4


## Concepto: caché por parámetros (no por prompt)

La forma más robusta de cachear en un agente con tools es **cachear el resultado de la tool** usando
una clave derivada de sus **parámetros estructurados** (los que el agente extrae).

Ventajas:
- Dos prompts distintos que implican el mismo cálculo terminan en la misma clave
- Evitas caches frágiles basados en texto libre
- Puedes añadir TTL e invalidación por versión de herramienta

In [2]:
class TTLCache:
    def __init__(self, ttl_seconds: float, max_entries: int = 256):
        self.ttl_seconds = float(ttl_seconds)
        self.max_entries = int(max_entries)
        self._store: dict[str, tuple[float, object]] = {}
        self.hits = 0
        self.misses = 0

    def get(self, key: str):
        now = time.time()
        item = self._store.get(key)
        if item is None:
            self.misses += 1
            return None

        expires_at, value = item
        if now >= expires_at:
            # Caducado
            self._store.pop(key, None)
            self.misses += 1
            return None

        self.hits += 1
        return value

    def set(self, key: str, value):
        if len(self._store) >= self.max_entries:
            # Evicción simple: elimina un elemento cualquiera (suficiente para workshop)
            self._store.pop(next(iter(self._store)))
        self._store[key] = (time.time() + self.ttl_seconds, value)

    def stats(self) -> str:
        return f"hits={self.hits}, misses={self.misses}, entries={len(self._store)}"

CACHE_VERSION = "v1"
CACHE_NAVEGACION = TTLCache(ttl_seconds=8, max_entries=128)
print("✅ TTLCache inicializado (TTL=8s)")

✅ TTLCache inicializado (TTL=8s)


## Tool: cálculo warp con simulación de coste (y caché)

Esta tool hace dos cosas:
- **Simula coste** con `time.sleep()` (como si llamara a una API o motor de cálculo)
- **Cachea** por parámetros para que llamadas equivalentes sean instantáneas

In [3]:
def _make_cache_key(operacion: str, factor_warp: float, distancia_anos_luz: float) -> str:
    payload = {
        "v": CACHE_VERSION,
        "op": str(operacion).strip().lower(),
        # Canonicalizamos floats para evitar claves distintas por 6 vs 6.0
        "warp": round(float(factor_warp), 3),
        "dist": round(float(distancia_anos_luz), 6),
    }
    blob = json.dumps(payload, sort_keys=True, ensure_ascii=False)
    return hashlib.sha256(blob.encode("utf-8")).hexdigest()

def calcular_trayectoria_warp_cacheada(
    operacion: Annotated[str, Field(description="La operación: calcular_velocidad, calcular_distancia, calcular_tiempo, o calcular_energia")],
    factor_warp: Annotated[float, Field(description="Factor warp (1-9.99)")],
    distancia_anos_luz: Annotated[float, Field(description="Distancia en años luz")],
) -> float:
    """Cálculos warp con caching por parámetros."""

    key = _make_cache_key(operacion, factor_warp, distancia_anos_luz)
    cached = CACHE_NAVEGACION.get(key)
    if cached is not None:
        print(f"⚡ [CACHE HIT] {operacion} warp={factor_warp} dist={distancia_anos_luz} | {CACHE_NAVEGACION.stats()}")
        return float(cached)

    print(f"🐢 [CACHE MISS] Ejecutando tool... {operacion} warp={factor_warp} dist={distancia_anos_luz}")
    # Simular coste de cálculo / llamada a servicio externo
    time.sleep(1.5)

    # Fórmula simplificada
    if operacion == "calcular_velocidad":
        resultado = factor_warp ** 3
    elif operacion == "calcular_distancia":
        resultado = distancia_anos_luz * (factor_warp ** 3)
    elif operacion == "calcular_tiempo":
        velocidad_efectiva = factor_warp ** 3
        resultado = 0 if velocidad_efectiva == 0 else (distancia_anos_luz / velocidad_efectiva) * 365.25
    elif operacion == "calcular_energia":
        resultado = (factor_warp ** 4) * distancia_anos_luz
    else:
        resultado = 0

    CACHE_NAVEGACION.set(key, float(resultado))
    print(f"💾 [CACHE STORE] guardado | {CACHE_NAVEGACION.stats()}")
    return float(resultado)

print("✅ Tool definida: calcular_trayectoria_warp_cacheada")

✅ Tool definida: calcular_trayectoria_warp_cacheada


## Registrar la tool en un agente

Instrucción importante: forzamos que el agente use la tool para preguntas de navegación, para que el caché se vea en el log.

In [4]:
agent = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=base_url,
        api_key=api_key,
        model_id=model_id,
    ),
    name="ComputadoraEnterpriseCache",
    instructions=(
        "Eres la computadora de navegación de la USS Enterprise-D. ",
        "SIEMPRE usa la herramienta calcular_trayectoria_warp_cacheada para cálculos warp. ",
        "Responde en una sola línea, en español, con el resultado numérico y la unidad cuando aplique.",
    ),
    tools=calcular_trayectoria_warp_cacheada,
)
print("✅ Agente creado con tool cacheada")

✅ Agente creado con tool cacheada


## Demo: misma pregunta, dos veces (cache hit visible)

Medimos el tiempo total de `agent.run()` y observamos los logs de la tool:
- Primera vez: `CACHE MISS` + ~1.5s
- Segunda vez: `CACHE HIT` + casi instantáneo

In [5]:
async def medir(consulta: str):
    t0 = time.perf_counter()
    r = await agent.run(consulta)
    dt = time.perf_counter() - t0
    print(f"⏱️ Tiempo total: {dt:.2f}s")
    print(f"🖥️ Respuesta: {r.text}")
    return dt

consulta_a = "Computadora, ¿cuánto tardaremos en llegar a Vulcano a warp 6 si está a 16 años luz?"
consulta_b = "Dime el tiempo de viaje a 16 años luz viajando a warp 6"

print("\n--- PRIMERA LLAMADA (espera CACHE MISS) ---")
await medir(consulta_a)

print("\n--- SEGUNDA LLAMADA (mismo cálculo, distinto prompt; espera CACHE HIT) ---")
await medir(consulta_b)


--- PRIMERA LLAMADA (espera CACHE MISS) ---
🐢 [CACHE MISS] Ejecutando tool... calcular_tiempo warp=6.0 dist=16.0
💾 [CACHE STORE] guardado | hits=0, misses=1, entries=1
⏱️ Tiempo total: 3.89s
🖥️ Respuesta: A velocidad warp 6, tomaría aproximadamente 27.06 días llegar a Vulcano, que está a 16 años luz de distancia.

--- SEGUNDA LLAMADA (mismo cálculo, distinto prompt; espera CACHE HIT) ---
⚡ [CACHE HIT] calcular_tiempo warp=6.0 dist=16.0 | hits=1, misses=1, entries=1
⏱️ Tiempo total: 1.20s
🖥️ Respuesta: Viajando a warp 6, el tiempo estimado para recorrer 16 años luz sería aproximadamente 27.06 días.


1.2038841999601573

## Demo: caducidad (TTL)

El cache tiene TTL=8s. Si esperamos lo suficiente, volvemos a ver `CACHE MISS`.

In [6]:
print("\nEsperando 9s para forzar expiración...")
time.sleep(9)
print("\n--- TERCERA LLAMADA (después del TTL; espera CACHE MISS) ---")
await medir(consulta_a)
print("\n📊 Stats finales cache: " + CACHE_NAVEGACION.stats())


Esperando 9s para forzar expiración...

--- TERCERA LLAMADA (después del TTL; espera CACHE MISS) ---
🐢 [CACHE MISS] Ejecutando tool... calcular_tiempo warp=6.0 dist=16.0
💾 [CACHE STORE] guardado | hits=1, misses=2, entries=1
⏱️ Tiempo total: 3.31s
🖥️ Respuesta: A warp 6, tardaremos aproximadamente 27.06 días en llegar a Vulcano, que está a 16 años luz de distancia.

📊 Stats finales cache: hits=1, misses=2, entries=1


## Takeaways (para contar en el workshop)

- Cachear **tools** es más estable que cachear prompts (la clave es estructurada)
- TTL evita respuestas obsoletas y sirve como invalidación simple
- Las métricas (hits/misses) te dicen si el caché aporta valor
- En producción, el siguiente paso es caché distribuido (Redis) + claves por tenant/usuario + trazas